Image Addition

In [2]:
import cv2 as cv
import numpy as np
x = np.uint8([250])
y = np.uint8([10])
print( cv.add(x,y) ) # 250+10 = 260 => 255
print( x+y )          # 250+10 = 260 % 256 = 4

[[255]]
[4]


Mezcla de imágenes
Esto también es una adición de imágenes, pero se les da diferentes pesos para dar una sensación de fusión o transparencia.
Aquí tomé dos imágenes para combinarlas. A la primera imagen se le asigna un peso de 0,7 y a la segunda imagen se le asigna 0,3. cv.addWeighted() aplica la siguiente ecuación a la imagen:

In [ ]:
img1 = cv.imread('resources/ml.png')
img2 = cv.imread('resources/opencv-logo.png')
print("img1 shape:", img1.shape)
print("img2 shape:", img2.shape)

assert img1 is not None, "file could not be read, check with os.path.exists()"
assert img2 is not None, "file could not be read, check with os.path.exists()"
dst = cv.addWeighted(img1,0.7,img2,0.3,0)
cv.imshow('dst',dst)
cv.waitKey(0)
cv.destroyAllWindows()

Operaciones bit a bit
Esto incluye las operaciones bit a bit AND, OR, NOT y XOR. Serán muy útiles al extraer cualquier parte de la imagen (como veremos en los próximos capítulos), definir y trabajar con ROI no rectangulares, etc. A continuación veremos un ejemplo de cómo cambiar una región particular de una imagen. .

Quiero poner el logo de OpenCV encima de una imagen. Si agrego dos imágenes, cambiará el color. Si los difumino, obtengo un efecto transparente. Pero quiero que sea opaco. Si fuera una región rectangular, podría usar el ROI como hicimos en el capítulo anterior. Pero el logotipo de OpenCV no tiene forma rectangular. Entonces puedes hacerlo con operaciones bit a bit como se muestra a continuación

In [5]:
# Load two images
img1 = cv.imread('resources/messi5.jpg')
img2 = cv.imread('resources/opencv-logo-white.png')
assert img1 is not None, "file could not be read, check with os.path.exists()"
assert img2 is not None, "file could not be read, check with os.path.exists()"
# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols]
# Now create a mask of logo and create its inverse mask also
img2gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2gray, 10, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)
# Now black-out the area of logo in ROI
img1_bg = cv.bitwise_and(roi,roi,mask = mask_inv)
# Take only region of logo from logo image.
img2_fg = cv.bitwise_and(img2,img2,mask = mask)
# Put logo in ROI and modify the main image
dst = cv.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst
cv.imshow('res',img1)
cv.waitKey(0)
cv.destroyAllWindows()

Exercise

In [ ]:
import cv2
import os
import glob
from threading import Timer

In [ ]:
def read_images(folder_path):
    """
    Read all images from a folder and return a list of image paths.
    """
    image_files = sorted(glob.glob(os.path.join(folder_path, '*.jpg')))  # Change the extension if needed
    return [cv2.imread(img) for img in image_files]

In [ ]:
def blend_images(img1, img2, alpha):
    """
    Blend two images using cv2.addWeighted.
    """
    return cv2.addWeighted(img1, alpha, img2, 1 - alpha, 0)

In [ ]:
def show_slideshow(images, delay_between_images=2000):
    """
    Display a slideshow with smooth transitions between images.
    """
    window_name = 'Slideshow'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    current_index = 0

    def transition():
        nonlocal current_index
        img1 = images[current_index]
        current_index = (current_index + 1) % len(images)
        img2 = images[current_index]

        alpha = 0.0
        while alpha <= 1.0:
            blended = blend_images(img1, img2, alpha)
            cv2.imshow(window_name, blended)
            alpha += 0.01  # Adjust the step size for smoother transitions
            cv2.waitKey(10)

        timer = Timer(delay_between_images / 1000.0, transition)
        timer.start()

    transition()  # Start the initial transition

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
folder_path = 'resources'  # Replace with the path to your image folder
images = read_images(folder_path)

if not images:
    print("No images found in the specified folder.")
else:
    show_slideshow(images)